In [2]:
import pandas as pd
import re
import numpy as np
import scipy.stats as stats


In [3]:
working_dir = "C:\\Users\\Joshua\\Documents\\GitSources\\TabletopGames_StirFry18\\SF18Experiments\\100RUNS"
action_chosen_dataframe = pd.read_csv(working_dir+'\\ACTION_CHOSEN.csv')


In [4]:
action_chosen_dataframe

,GameSeed,GameName,Round,Event,Tick,Turn,PlayerCount,GameID,CookList(Cooked with -),StateSpace(Size),...,GameScore(LeaderGap),GameScore(PlayerName-0),GameScore(Player-1),GameScore(Average),GameScore(Player-0),GameScore(Player-3),GameScore(Player-2),GameScore(PlayerName-1),GameScore(PlayerName-2),GameScore(PlayerName-3)
0,2034470281,StirFry18,0,ACTION_CHOSEN,0,0,4,2,NaN,22,...,0.0,mcts,0.0,0.0,0.0,0.0,0.0,mcts,mcts,mcts
1,2034470281,StirFry18,0,ACTION_CHOSEN,1,0,4,2,NaN,22,...,0.0,mcts,0.0,0.0,0.0,0.0,0.0,mcts,mcts,mcts
2,2034470281,StirFry18,0,ACTION_CHOSEN,2,1,4,2,MUSHROOMS-NOODLES-SOY_SAUCE-,22,...,0.0,mcts,0.0,0.0,0.0,0.0,0.0,mcts,mcts,mcts
3,2034470281,StirFry18,0,ACTION_CHOSEN,3,1,4,2,NaN,22,...,4.0,mcts,4.0,1.0,0.0,0.0,0.0,mcts,mcts,mcts
4,2034470281,StirFry18,0,ACTION_CHOSEN,4,1,4,2,NaN,22,...,4.0,mcts,4.0,1.0,0.0,0.0,0.0,mcts,mcts,mcts
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13372,470519093,StirFry18,0,ACTION_CHOSEN,150,47,4,101,NaN,22,...,1.0,mcts,39.0,37.0,37.0,34.0,38.0,mcts,mcts,mcts
13373,470519093,StirFry18,0,ACTION_CHOSEN,151,48,4,101,NaN,22,...,1.0,mcts,39.0,37.0,37.0,34.0,38.0,mcts,mcts,mcts
13374,470519093,StirFry18,0,ACTION_CHOSEN,152,48,4,101,CHICKEN-GREEN_ONION-NOODLES-SHRIMP-SOY_SAUCE-,22,...,1.0,mcts,39.0,37.0,37.0,34.0,38.0,mcts,mcts,mcts
13375,470519093,StirFry18,0,ACTION_CHOSEN,153,48,4,101,NaN,22,...,12.0,mcts,39.0,40.5,51.0,34.0,38.0,mcts,mcts,mcts


In [5]:
# Simplify the dataframe to keep only the specified columns
simplified_df = action_chosen_dataframe[[
    'GameSeed',  # Unique identifier for each game
    'Actions(Player-0)',  # Actions taken by player 0
    'Actions(Player-1)',  # Actions taken by player 1
    'Actions(Player-2)',  # Actions taken by player 2
    'Actions(Player-3)',  # Actions taken by player 3
    'GameScore(Player-0)',  # Score for player 0
    'GameScore(Player-1)',  # Score for player 1
    'GameScore(Player-2)',  # Score for player 2
    'GameScore(Player-3)',  # Score for player 3
    'CookList(Cooked with -)'  # Ingredients used in cooking actions
]]

# Display the first few rows of the simplified dataframe
simplified_df

,GameSeed,Actions(Player-0),Actions(Player-1),Actions(Player-2),Actions(Player-3),GameScore(Player-0),GameScore(Player-1),GameScore(Player-2),GameScore(Player-3),CookList(Cooked with -)
0,2034470281,pass,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN
1,2034470281,pass,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN
2,2034470281,NaN,"Cook with ids: 41, 43, 47,",NaN,NaN,0.0,0.0,0.0,0.0,MUSHROOMS-NOODLES-SOY_SAUCE-
3,2034470281,NaN,pass,NaN,NaN,0.0,4.0,0.0,0.0,NaN
4,2034470281,NaN,pass,NaN,NaN,0.0,4.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
13372,470519093,NaN,NaN,NaN,pass,37.0,39.0,38.0,34.0,NaN
13373,470519093,Discard Protein: 16918,NaN,NaN,NaN,37.0,39.0,38.0,34.0,NaN
13374,470519093,"Cook with ids: 16930, 16917, 16919, 16922, 16926,",NaN,NaN,NaN,37.0,39.0,38.0,34.0,CHICKEN-GREEN_ONION-NOODLES-SHRIMP-SOY_SAUCE-
13375,470519093,pass,NaN,NaN,NaN,51.0,39.0,38.0,34.0,NaN


In [6]:
# Create a new dataframe to find winners by game seed
winners_df = simplified_df.groupby('GameSeed').agg({
    'GameScore(Player-0)': 'max',
    'GameScore(Player-1)': 'max',
    'GameScore(Player-2)': 'max',
    'GameScore(Player-3)': 'max'
})

# Determine the winner for each game seed
def get_winner(row):
    for player in range(4):
        if row[f'GameScore(Player-{player})'] > 50:
            return player
    return -1  # No winner found

winners_df['Winner'] = winners_df.apply(get_winner, axis=1)

# Merge the winner information back to the original dataframe
simplified_df = simplified_df.merge(
    winners_df[['Winner']], 
    left_on='GameSeed', 
    right_index=True
)

# Display the first few rows of the updated dataframe
simplified_df

,GameSeed,Actions(Player-0),Actions(Player-1),Actions(Player-2),Actions(Player-3),GameScore(Player-0),GameScore(Player-1),GameScore(Player-2),GameScore(Player-3),CookList(Cooked with -),Winner
0,2034470281,pass,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0
1,2034470281,pass,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0
2,2034470281,NaN,"Cook with ids: 41, 43, 47,",NaN,NaN,0.0,0.0,0.0,0.0,MUSHROOMS-NOODLES-SOY_SAUCE-,0
3,2034470281,NaN,pass,NaN,NaN,0.0,4.0,0.0,0.0,NaN,0
4,2034470281,NaN,pass,NaN,NaN,0.0,4.0,0.0,0.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
13372,470519093,NaN,NaN,NaN,pass,37.0,39.0,38.0,34.0,NaN,0
13373,470519093,Discard Protein: 16918,NaN,NaN,NaN,37.0,39.0,38.0,34.0,NaN,0
13374,470519093,"Cook with ids: 16930, 16917, 16919, 16922, 16926,",NaN,NaN,NaN,37.0,39.0,38.0,34.0,CHICKEN-GREEN_ONION-NOODLES-SHRIMP-SOY_SAUCE-,0
13375,470519093,pass,NaN,NaN,NaN,51.0,39.0,38.0,34.0,NaN,0


In [7]:
# Create a function to determine which player took the action in each row
def get_active_player(row):
    for player in range(4):
        action_col = f'Actions(Player-{player})'
        if pd.notna(row[action_col]) and row[action_col] != '':
            return player
    return -1  # No player found with an action

# Create a function to get the action name from the player's action column
def get_action_name(row):
    player = row['ActivePlayer']
    if player != -1:
        action_col = f'Actions(Player-{player})'
        return row[action_col]
    return ''

# Add the ActivePlayer column
simplified_df['ActivePlayer'] = simplified_df.apply(get_active_player, axis=1)

# Add the ActionTaken column
simplified_df['ActionTaken'] = simplified_df.apply(get_action_name, axis=1)

# Display the first few rows of the updated dataframe
simplified_df

,GameSeed,Actions(Player-0),Actions(Player-1),Actions(Player-2),Actions(Player-3),GameScore(Player-0),GameScore(Player-1),GameScore(Player-2),GameScore(Player-3),CookList(Cooked with -),Winner,ActivePlayer,ActionTaken
0,2034470281,pass,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,0,pass
1,2034470281,pass,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,0,pass
2,2034470281,NaN,"Cook with ids: 41, 43, 47,",NaN,NaN,0.0,0.0,0.0,0.0,MUSHROOMS-NOODLES-SOY_SAUCE-,0,1,"Cook with ids: 41, 43, 47,"
3,2034470281,NaN,pass,NaN,NaN,0.0,4.0,0.0,0.0,NaN,0,1,pass
4,2034470281,NaN,pass,NaN,NaN,0.0,4.0,0.0,0.0,NaN,0,1,pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13372,470519093,NaN,NaN,NaN,pass,37.0,39.0,38.0,34.0,NaN,0,3,pass
13373,470519093,Discard Protein: 16918,NaN,NaN,NaN,37.0,39.0,38.0,34.0,NaN,0,0,Discard Protein: 16918
13374,470519093,"Cook with ids: 16930, 16917, 16919, 16922, 16926,",NaN,NaN,NaN,37.0,39.0,38.0,34.0,CHICKEN-GREEN_ONION-NOODLES-SHRIMP-SOY_SAUCE-,0,0,"Cook with ids: 16930, 16917, 16919, 16922, 16926,"
13375,470519093,pass,NaN,NaN,NaN,51.0,39.0,38.0,34.0,NaN,0,0,pass


In [8]:
# Get the final scores for each player in each game
final_scores = simplified_df.groupby('GameSeed').agg({
    'GameScore(Player-0)': 'max',
    'GameScore(Player-1)': 'max',
    'GameScore(Player-2)': 'max',
    'GameScore(Player-3)': 'max'
})

# Rename the columns to indicate these are final scores
final_scores = final_scores.rename(columns={
    'GameScore(Player-0)': 'FinalScore_Player0',
    'GameScore(Player-1)': 'FinalScore_Player1',
    'GameScore(Player-2)': 'FinalScore_Player2',
    'GameScore(Player-3)': 'FinalScore_Player3'
})

# Merge the final scores back to the original dataframe
simplified_df = simplified_df.merge(
    final_scores,
    left_on='GameSeed',
    right_index=True
)

# Display the first few rows of the updated dataframe
simplified_df

,GameSeed,Actions(Player-0),Actions(Player-1),Actions(Player-2),Actions(Player-3),GameScore(Player-0),GameScore(Player-1),GameScore(Player-2),GameScore(Player-3),CookList(Cooked with -),Winner,ActivePlayer,ActionTaken,FinalScore_Player0,FinalScore_Player1,FinalScore_Player2,FinalScore_Player3
0,2034470281,pass,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,0,pass,51.0,37.0,45.0,47.0
1,2034470281,pass,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,0,pass,51.0,37.0,45.0,47.0
2,2034470281,NaN,"Cook with ids: 41, 43, 47,",NaN,NaN,0.0,0.0,0.0,0.0,MUSHROOMS-NOODLES-SOY_SAUCE-,0,1,"Cook with ids: 41, 43, 47,",51.0,37.0,45.0,47.0
3,2034470281,NaN,pass,NaN,NaN,0.0,4.0,0.0,0.0,NaN,0,1,pass,51.0,37.0,45.0,47.0
4,2034470281,NaN,pass,NaN,NaN,0.0,4.0,0.0,0.0,NaN,0,1,pass,51.0,37.0,45.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13372,470519093,NaN,NaN,NaN,pass,37.0,39.0,38.0,34.0,NaN,0,3,pass,51.0,39.0,38.0,34.0
13373,470519093,Discard Protein: 16918,NaN,NaN,NaN,37.0,39.0,38.0,34.0,NaN,0,0,Discard Protein: 16918,51.0,39.0,38.0,34.0
13374,470519093,"Cook with ids: 16930, 16917, 16919, 16922, 16926,",NaN,NaN,NaN,37.0,39.0,38.0,34.0,CHICKEN-GREEN_ONION-NOODLES-SHRIMP-SOY_SAUCE-,0,0,"Cook with ids: 16930, 16917, 16919, 16922, 16926,",51.0,39.0,38.0,34.0
13375,470519093,pass,NaN,NaN,NaN,51.0,39.0,38.0,34.0,NaN,0,0,pass,51.0,39.0,38.0,34.0


In [9]:
# Calculate the highest score for each game
simplified_df['HighestScore'] = simplified_df[['FinalScore_Player0', 'FinalScore_Player1', 
                                              'FinalScore_Player2', 'FinalScore_Player3']].max(axis=1)

# Calculate the score weight for each player
for player in range(4):
    simplified_df[f'ScoreWeight_Player{player}'] = simplified_df[f'FinalScore_Player{player}'] / simplified_df['HighestScore']

# Round the score weights to 3 decimal places for readability
for player in range(4):
    simplified_df[f'ScoreWeight_Player{player}'] = simplified_df[f'ScoreWeight_Player{player}'].round(3)

# Optionally, drop the temporary HighestScore column if not needed
simplified_df = simplified_df.drop(columns=['HighestScore'])

# Display the first few rows of the updated dataframe
simplified_df

,GameSeed,Actions(Player-0),Actions(Player-1),Actions(Player-2),Actions(Player-3),GameScore(Player-0),GameScore(Player-1),GameScore(Player-2),GameScore(Player-3),CookList(Cooked with -),...,ActivePlayer,ActionTaken,FinalScore_Player0,FinalScore_Player1,FinalScore_Player2,FinalScore_Player3,ScoreWeight_Player0,ScoreWeight_Player1,ScoreWeight_Player2,ScoreWeight_Player3
0,2034470281,pass,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,...,0,pass,51.0,37.0,45.0,47.0,1.0,0.725,0.882,0.922
1,2034470281,pass,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,...,0,pass,51.0,37.0,45.0,47.0,1.0,0.725,0.882,0.922
2,2034470281,NaN,"Cook with ids: 41, 43, 47,",NaN,NaN,0.0,0.0,0.0,0.0,MUSHROOMS-NOODLES-SOY_SAUCE-,...,1,"Cook with ids: 41, 43, 47,",51.0,37.0,45.0,47.0,1.0,0.725,0.882,0.922
3,2034470281,NaN,pass,NaN,NaN,0.0,4.0,0.0,0.0,NaN,...,1,pass,51.0,37.0,45.0,47.0,1.0,0.725,0.882,0.922
4,2034470281,NaN,pass,NaN,NaN,0.0,4.0,0.0,0.0,NaN,...,1,pass,51.0,37.0,45.0,47.0,1.0,0.725,0.882,0.922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13372,470519093,NaN,NaN,NaN,pass,37.0,39.0,38.0,34.0,NaN,...,3,pass,51.0,39.0,38.0,34.0,1.0,0.765,0.745,0.667
13373,470519093,Discard Protein: 16918,NaN,NaN,NaN,37.0,39.0,38.0,34.0,NaN,...,0,Discard Protein: 16918,51.0,39.0,38.0,34.0,1.0,0.765,0.745,0.667
13374,470519093,"Cook with ids: 16930, 16917, 16919, 16922, 16926,",NaN,NaN,NaN,37.0,39.0,38.0,34.0,CHICKEN-GREEN_ONION-NOODLES-SHRIMP-SOY_SAUCE-,...,0,"Cook with ids: 16930, 16917, 16919, 16922, 16926,",51.0,39.0,38.0,34.0,1.0,0.765,0.745,0.667
13375,470519093,pass,NaN,NaN,NaN,51.0,39.0,38.0,34.0,NaN,...,0,pass,51.0,39.0,38.0,34.0,1.0,0.765,0.745,0.667


In [10]:
# Complete combos dictionary based on the provided list
combos_dict = {
    # Shrimp combos
    "Shrimp-Ginger": ["Shrimp", "Ginger"],
    "Shrimp-Soy Sauce-Ginger": ["Shrimp", "Soy Sauce", "Ginger"],
    "Shrimp-Green Onion": ["Shrimp", "Green Onion"],
    
    # Pork combos
    "Pork-Mushrooms": ["Pork", "Mushrooms"],
    "Pork-Ginger": ["Pork", "Ginger"],
    
    # Chicken combos
    "Chicken-Green Onion": ["Chicken", "Green Onion"],
    "Chicken-Ginger-Green Onion": ["Chicken", "Ginger", "Green Onion"],
    "Chicken-Mushrooms": ["Chicken", "Mushrooms"],
    
    # Soy Sauce combos
    "Soy Sauce-Green Onion": ["Soy Sauce", "Green Onion"],
    "Soy Sauce-Ginger": ["Soy Sauce", "Ginger"]
}

def identify_combos_and_individuals(cook_list):
    if pd.isna(cook_list) or cook_list == "":
        return [], []
    
    # Split the ingredients and standardize format
    ingredients = [ing.strip().upper() for ing in cook_list.split("-")]
    ingredients = [ing for ing in ingredients if ing]  # Remove empty strings
    
    # Create a standardized version of ingredients for matching (replace underscores with spaces)
    std_ingredients = [ing.replace("_", " ") for ing in ingredients]
    
    found_combos = []
    
    # Check for all possible combos
    for combo_name, combo_ingredients in combos_dict.items():
        # Convert combo ingredients to uppercase for case-insensitive comparison
        combo_ingredients_upper = [ing.upper() for ing in combo_ingredients]
        
        # Check if all ingredients in this combo are present
        if all(ing in std_ingredients for ing in combo_ingredients_upper):
            found_combos.append(combo_name)
    
    # For individual ingredients, we need to check which ones weren't used in any combo
    used_in_combos = set()
    for combo_name in found_combos:
        for ing in combos_dict[combo_name]:
            used_in_combos.add(ing.upper())
    
    # Find individual ingredients
    individual_ingredients = [ing for ing in ingredients if ing.replace("_", " ") not in used_in_combos]
    
    return found_combos, individual_ingredients

# Apply the function to create new columns
simplified_df[['Combos', 'Individual_Ingredients']] = simplified_df.apply(
    lambda row: pd.Series(identify_combos_and_individuals(row['CookList(Cooked with -)'])), 
    axis=1
)

# Display the first few rows to verify
simplified_df[['CookList(Cooked with -)', 'Combos', 'Individual_Ingredients']]

,CookList(Cooked with -),Combos,Individual_Ingredients
0,NaN,[],[]
1,NaN,[],[]
2,MUSHROOMS-NOODLES-SOY_SAUCE-,[],"[MUSHROOMS, NOODLES, SOY_SAUCE]"
3,NaN,[],[]
4,NaN,[],[]
...,...,...,...
13372,NaN,[],[]
13373,NaN,[],[]
13374,CHICKEN-GREEN_ONION-NOODLES-SHRIMP-SOY_SAUCE-,"[Shrimp-Green Onion, Chicken-Green Onion, Soy ...",[NOODLES]
13375,NaN,[],[]


In [11]:

# Create dictionaries to track scores for statistical analysis
combo_scores = {}
ingredient_scores = {}
combo_counts = {}
ingredient_counts = {}

# Process each row in the dataframe
for _, row in simplified_df.iterrows():
    # Skip if no cook action or missing data
    if pd.isna(row['CookList(Cooked with -)']) or row['CookList(Cooked with -)'] == "":
        continue
    
    # Get the active player who performed the cook action
    active_player = row['ActivePlayer']
    
    # Get the score weight for the active player
    score_weight_col = f'ScoreWeight_Player{active_player}'
    if score_weight_col not in row or pd.isna(row[score_weight_col]):
        continue
    
    score_weight = row[score_weight_col]
    
    # Process combos
    for combo in row['Combos']:
        if combo not in combo_scores:
            combo_scores[combo] = []
            combo_counts[combo] = 0
        combo_scores[combo].append(score_weight)
        combo_counts[combo] += 1
    
    # Process individual ingredients
    for ingredient in row['Individual_Ingredients']:
        if ingredient not in ingredient_scores:
            ingredient_scores[ingredient] = []
            ingredient_counts[ingredient] = 0
        ingredient_scores[ingredient].append(score_weight)
        ingredient_counts[ingredient] += 1

# Calculate statistics
combo_stats = {}
for combo, scores in combo_scores.items():
    if len(scores) > 1:  # Need at least 2 samples for std error
        mean = np.mean(scores)
        std_error = stats.sem(scores)
        combo_stats[combo] = {
            'Mean': mean,
            'StdError': std_error,
            'Count': len(scores),
            'CI_Lower': mean - 1.96 * std_error,  # 95% confidence interval
            'CI_Upper': mean + 1.96 * std_error
        }
    elif len(scores) == 1:  # Only one sample, can't calculate std error
        mean = scores[0]
        combo_stats[combo] = {
            'Mean': mean,
            'StdError': None,
            'Count': 1,
            'CI_Lower': None,
            'CI_Upper': None
        }

ingredient_stats = {}
for ing, scores in ingredient_scores.items():
    if len(scores) > 1:  # Need at least 2 samples for std error
        mean = np.mean(scores)
        std_error = stats.sem(scores)
        ingredient_stats[ing] = {
            'Mean': mean,
            'StdError': std_error,
            'Count': len(scores),
            'CI_Lower': mean - 1.96 * std_error,  # 95% confidence interval
            'CI_Upper': mean + 1.96 * std_error
        }
    elif len(scores) == 1:  # Only one sample, can't calculate std error
        mean = scores[0]
        ingredient_stats[ing] = {
            'Mean': mean,
            'StdError': None,
            'Count': 1,
            'CI_Lower': None,
            'CI_Upper': None
        }

# Create DataFrames for better visualization
combo_df = pd.DataFrame.from_dict(combo_stats, orient='index')
combo_df.reset_index(inplace=True)
combo_df.rename(columns={'index': 'Combo'}, inplace=True)
combo_df = combo_df.sort_values('Mean', ascending=False)

ingredient_df = pd.DataFrame.from_dict(ingredient_stats, orient='index')
ingredient_df.reset_index(inplace=True)
ingredient_df.rename(columns={'index': 'Ingredient'}, inplace=True)
ingredient_df = ingredient_df.sort_values('Mean', ascending=False)

# Display the results
print("Combo Average Score Weights with Error Margins:")
print(combo_df)
print("\nIndividual Ingredient Average Score Weights with Error Margins:")
print(ingredient_df)


Combo Average Score Weights with Error Margins:
                        Combo      Mean  StdError  Count  CI_Lower  CI_Upper
9  Chicken-Ginger-Green Onion  0.882218  0.015323     78  0.852185  0.912251
0               Shrimp-Ginger  0.841291  0.013087    148  0.815641  0.866940
1              Pork-Mushrooms  0.836979  0.011036    241  0.815349  0.858609
5                 Pork-Ginger  0.836959  0.012666    171  0.812134  0.861784
6         Chicken-Green Onion  0.834477  0.011956    193  0.811043  0.857910
4           Chicken-Mushrooms  0.830988  0.010249    256  0.810901  0.851076
8          Shrimp-Green Onion  0.826549  0.014231    133  0.798656  0.854441
7     Shrimp-Soy Sauce-Ginger  0.824297  0.022057     64  0.781065  0.867529
2       Soy Sauce-Green Onion  0.789436  0.008114    512  0.773532  0.805339
3            Soy Sauce-Ginger  0.779331  0.008508    474  0.762655  0.796008

Individual Ingredient Average Score Weights with Error Margins:
    Ingredient      Mean  StdError  Coun

In [12]:
# Create a combined DataFrame with a new column indicating the type (Combo or Ingredient)
# First, add a 'Type' column to each DataFrame
combo_df['Type'] = 'Combo'
ingredient_df['Type'] = 'Ingredient'

# Rename columns to be consistent
combo_df.rename(columns={'Combo': 'Item'}, inplace=True)
ingredient_df.rename(columns={'Ingredient': 'Item'}, inplace=True)

# Combine the DataFrames
combined_df = pd.concat([combo_df, ingredient_df], ignore_index=True)

# Sort by Mean score in descending order
combined_df = combined_df.sort_values('Mean', ascending=False)

print("\nTop 10 Items Overall:")
print(combined_df)


Top 10 Items Overall:
                          Item      Mean  StdError  Count  CI_Lower  CI_Upper  \
0   Chicken-Ginger-Green Onion  0.882218  0.015323     78  0.852185  0.912251   
1                Shrimp-Ginger  0.841291  0.013087    148  0.815641  0.866940   
2               Pork-Mushrooms  0.836979  0.011036    241  0.815349  0.858609   
3                  Pork-Ginger  0.836959  0.012666    171  0.812134  0.861784   
4          Chicken-Green Onion  0.834477  0.011956    193  0.811043  0.857910   
5            Chicken-Mushrooms  0.830988  0.010249    256  0.810901  0.851076   
10                      SHRIMP  0.827651  0.014931    126  0.798386  0.856916   
6           Shrimp-Green Onion  0.826549  0.014231    133  0.798656  0.854441   
7      Shrimp-Soy Sauce-Ginger  0.824297  0.022057     64  0.781065  0.867529   
11                        PORK  0.810429  0.016422    105  0.778241  0.842616   
12                     CHICKEN  0.805144  0.017299     97  0.771238  0.839051   
13   

In [14]:
# Save the combined ranking to a CSV file
output_filename = working_dir+'\\combined_ingredient_ranking.csv'
combined_df.to_csv(output_filename, index=False)
print(f"\nCombined ranking saved to '{output_filename}'")


Combined ranking saved to 'C:\Users\Joshua\Documents\GitSources\TabletopGames_StirFry18\SF18Experiments\100RUNS\combined_ingredient_ranking.csv'
